**Import Dependencies** 

In [1]:
import os
import datetime
import openpyxl
import pandas as pd

**Define Process Functions**  

In [14]:
# Assumptions:
# 1. Source files are .xlsx
# 2. Source file names end with date (yyyyMMdd)

def get_source_file_names_for_dates(prefix: str, file_format: str, dates: list[datetime.date]) -> set[str]:
    return set(["{1}{2}.{3}".format(dir, prefix, date.strftime("%Y%m%d"), file_format) for date in dates])


def get_source_files_matching_names(dir: str, file_names: set[str]) -> list[os.path]:
    res = file_names.intersection(set(os.listdir(dir)))
    missing_files = file_names.difference(res)
    if (len(missing_files)>0):
        raise Exception(f"Can't find the following requested files: {', '.join(missing_files)}")
    return [os.path.join(dir, file_name) for file_name in res]


def get_source_file_highlighted_data(file_path: os.path) -> pd.DataFrame:
    wb = openpyxl.load_workbook(filename=str(file_path), read_only=True)
    highlighted_rows = []
    for row in range(wb.worksheets[0].min_row + 1, wb.worksheets[0].max_row + 1):
        for col in range(wb.worksheets[0].min_column, wb.worksheets[0].max_column + 1):
            c = wb.worksheets[0].cell(row, col)
            if c.fill.start_color.index != "00000000":
                highlighted_rows.append(row-2)
                break

    file_data = pd.read_excel(file_path)
    highlighed_data = pd.DataFrame(columns=file_data.columns)
    for highlighted_row_idx in highlighted_rows:
        highlighted_row = pd.Series(file_data.iloc[highlighted_row_idx].to_list(), index=highlighed_data.columns)
        highlighed_data = highlighed_data.append(highlighted_row, ignore_index=True)
    
    return highlighed_data


def get_source_files_highlighted_data(file_paths: list[os.path]) -> list[pd.DataFrame]:
    res = [get_source_file_highlighted_data(file_path) for file_path in file_paths]
    if (len(res)>0):
        base_cols = [col for col in res[0].columns]
        for df in res[1:]:
            check_cols = [col for col in df.columns]
            if len(base_cols) != len(check_cols):
                raise Exception("Files don't all have the same number of columns!")
            elif base_cols != check_cols:
                raise Exception("Files don't all have same column names!")
    return res


def gather_highlighted_rows(source_dir: str, source_file_prefix: str, target_dir: str, target_file_name: str, dates: list[datetime.date], overwrite: bool = False):
    file_format = "xlsx"
    expected_source_files = get_source_file_names_for_dates(source_file_prefix, file_format, dates)
    source_files = get_source_files_matching_names(source_dir, expected_source_files)
    source_dfs = get_source_files_highlighted_data(source_files)
    output_df = pd.concat(source_dfs, ignore_index=True)
    output_path = os.path.join(target_dir, f"{target_file_name}.{file_format}")
    if not overwrite and os.path.exists(output_path):
        raise Exception(f"Specified output file {str(output_path)} already exists!")
    output_df.to_excel(output_path, index=False)


**Run the Process**

In [15]:
# General params
source_dir = "SampleFiles"
source_file_prefix = "Test_"
target_dir = "SampleOutputFiles"
target_file_name = "TestOutput"

# Date param!
dates = [datetime.date(2021, 12, 1), datetime.date(2021, 12, 2), datetime.date(2021, 12, 3), datetime.date(2021, 12, 4)]


# Run process
gather_highlighted_rows(source_dir, source_file_prefix, target_dir, target_file_name, dates, overwrite=True)


**Test the Process**

In [16]:
### Test with output file that already exists (expect failure)
gather_highlighted_rows(source_dir, source_file_prefix, target_dir, target_file_name, dates)

Exception: Specified output file SampleOutputFiles\TestOutput.xlsx already exists!

In [17]:
### Test with dates that don't have files (expect failure)
dates = [datetime.date(2021, 12, 1), datetime.date(2021, 12, 2), datetime.date(2021, 12, 4), datetime.date(2021, 12, 5)]
gather_highlighted_rows(source_dir, source_file_prefix, target_dir, target_file_name, dates)

Exception: Can't find the following requested files: Test_20211205.xlsx

In [18]:
### Test with files that have different number of columns (expect failure)
dates = [datetime.date(2021, 12, 1), datetime.date(2021, 12, 2), datetime.date(2021, 12, 13)]
gather_highlighted_rows(source_dir, source_file_prefix, target_dir, target_file_name, dates)

Exception: Files don't all have the same number of columns!

In [19]:
### Test with files that have different column names (expect failure)
dates = [datetime.date(2021, 12, 1), datetime.date(2021, 12, 2), datetime.date(2021, 12, 14)]
gather_highlighted_rows(source_dir, source_file_prefix, target_dir, target_file_name, dates)

Exception: Files don't all have same column names!